<a href="https://colab.research.google.com/github/DucLeTrong/genetic_algorithm/blob/master/GA_WRSNs_Phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
random.seed( 30 )
import math
paras = {}
f = open('1.txt','r')
list_node = []
for x in f:
    list_node.append([float(i) for i in x.split(" ")])
paras["list_node"] = list_node
def calcualte_distances(list_node):
    distances =[[None]*len(list_node) for i in range(len(list_node))] 
    for i in range(len(list_node)):
        for j in range(len(list_node)):
            distances[i][j] = math.sqrt((list_node[i][0]-list_node[j][0])**2 + (list_node[i][1]-list_node[j][1])**2)

    # print(distances)
    return distances
paras['distances'] = calcualte_distances(list_node)
# distances[1][2]
paras['V'] = 5
paras['ALPHA'] = 0.5
paras['num_of_gens'] = len(paras['list_node'])
paras['size_popu'] = 100
paras["U"] = 5
paras['E_MC'] = 108000
paras["P_M"] = 1

In [0]:
# list_node = []
# f = open('2.txt','r')
# for x in f:
#     list_node.append([float(i) for i in x.split(" ")])
# paras['E_MC'] = 108000
# paras["P_M"] = 1
# paras['V'] = 5
# list_t_max= [i[3]- 540 for i in list_node]

In [0]:
f = open("1-result.txt",'r')
# print(f.readline())
# x = f.readline()
# x.split(' ')
charging_path = [int(i)-1 for i in f.readline().split(' ')[:-1]]

# charging_path 

In [0]:

S = 0
for i in range(1,len(charging_path)):
    S += paras['distances'][charging_path[i]][charging_path[i-1]]
total_charging_time = (paras['E_MC'] - (S*paras["P_M"] / paras['V'])) / paras["U"]
list_t_max= [(10800- 540)/(paras["U"] - list_node[i][2])/total_charging_time for i in charging_path]

distances = paras['distances']


In [0]:
def calcualte_fitness(chromosome,charging_path,distances,list_node):
    t_wait = [0]*len(charging_path)
    e_pre = [list_node[i][3] for i in charging_path]
    for i in range(1,len(charging_path)):
        t_wait[i] = t_wait[i-1] + chromosome[i-1]* + distances[charging_path[i-1]][charging_path[i]]/ paras['V'] /total_charging_time
        e_pre[i] -= t_wait[i] *list_node[charging_path[i]][2]
    fitness = sum([1 if i < 540 else 0 for i in e_pre ])
    return fitness


In [0]:
import numpy as np
class Individual:
    def __init__(self, input_value=0, init = False):
        if init:
            # charging_path = input_value
            self.chromosome = [0.0] * len(charging_path)
            mix_id = np.random.permutation(len(charging_path))

            for i in mix_id[0: len(charging_path)-1]:
                g = random.uniform(0,min(1-sum(self.chromosome),list_t_max[i]/total_charging_time))
                self.chromosome[i] = g
            self.chromosome[mix_id[-1]] = 1 -sum(self.chromosome)
            self.fitness_score = calcualte_fitness(self.chromosome,charging_path,distances,list_node)
        else:
            self.chromosome = input_value
            self.fitness_score = calcualte_fitness(self.chromosome,charging_path,distances,list_node)

        
    def get_fitness(self):
        return self.fitness_score

    def get_size(self):
        return len(self.chromosome)

    def mutate(self, list_t_max):
        if random.random() < 0.4:
            point1 = int(random.random() * self.get_size())
            point2 = int(random.random() * self.get_size())
            sigma1 = random.uniform(0,list_t_max[point1] - self.chromosome[point1])
            sigma2 = random.uniform(0,list_t_max[point2]-self.chromosome[point1])
            sigma = min(sigma1, sigma2)
            self.chromosome[point1] +=sigma
            self.chromosome[point2] -=sigma

  

In [0]:
class Population():
    def __init__(self, size_population):
        # self.size = size
        self.individuals = []
        for i in range(size_population):
            individual = Individual(init = True)
            self.individuals.append(individual)
        print(len(self.individuals))

    def get_size(self):
        return len(self.individuals)

    def get_individual(self, index):
      return self.individuals[index]

    def add_individual(self, individual):
        self.individuals.append(individual)


    def get_fittest(self):
        fittest = self.individuals[0]
        # print("len", self.get_size())
        for i in range(self.get_size()):
            # print("dasd",i,self.individuals[i].get_fitness())
            if fittest.get_fitness() > self.individuals[i].get_fitness():
                fittest = self.get_individual(i)
                
        # self.individuals = s
        return fittest

In [0]:
class GA:
    def crossover(self, ind1, ind2,list_t_max):
            return crossover(ind1, ind2 , list_t_max)

    def crossover_population(self, population, num_of_pair):
        for i in range(num_of_pair):
            j1 = random.randint(0,population.get_size()-1)
            j2 = random.randint(0,population.get_size()-1)
            while j1==j2:
                j2 = random.randint(0,population.get_size()-1)

            child1, child2 = self.crossover(population.individuals[j1], population.individuals[j2], list_t_max)
            population.add_individual(child1)
            population.add_individual(child2)

    def mutate_population(self, population):
        for individual in population.individuals:
            if random.random() < 0.2:
                individual.mutate(list_t_max)
    
    def natureSelection(self, population,size):
        population.individuals = sorted(population.individuals, key=Individual.get_fitness)[:size]


    def run(self, population, number_loop):
        for i in range(number_loop):
            self.crossover_population(population,100)
            self.mutate_population(population)
            self.natureSelection(population,250)
            print("loop", i, "best fitness",population.get_fittest().get_fitness())
        print("Best solotion",population.get_fittest().get_fitness())
        print("Best solotion",population.get_fittest().chromosome)

In [137]:
sum(population.get_fittest().chromosome)

1.0

In [135]:
random.seed(30)
paras['number_loop'] = 100
population = Population(paras['size_popu'])
ga = GA()
ga.run(population, paras['number_loop'])

100
loop 0 best fitness 0
loop 1 best fitness 0
loop 2 best fitness 0
loop 3 best fitness 0
loop 4 best fitness 0
loop 5 best fitness 0
loop 6 best fitness 0
loop 7 best fitness 0
loop 8 best fitness 0
loop 9 best fitness 0
loop 10 best fitness 0
loop 11 best fitness 0
loop 12 best fitness 0
loop 13 best fitness 0
loop 14 best fitness 0
loop 15 best fitness 0
loop 16 best fitness 0
loop 17 best fitness 0
loop 18 best fitness 0
loop 19 best fitness 0
loop 20 best fitness 0
loop 21 best fitness 0
loop 22 best fitness 0
loop 23 best fitness 0
loop 24 best fitness 0
loop 25 best fitness 0
loop 26 best fitness 0
loop 27 best fitness 0
loop 28 best fitness 0
loop 29 best fitness 0
loop 30 best fitness 0
loop 31 best fitness 0
loop 32 best fitness 0
loop 33 best fitness 0
loop 34 best fitness 0
loop 35 best fitness 0
loop 36 best fitness 0
loop 37 best fitness 0
loop 38 best fitness 0
loop 39 best fitness 0
loop 40 best fitness 0
loop 41 best fitness 0
loop 42 best fitness 0
loop 43 best fitn

In [90]:
parent1 = Individual([0.12,0.23,0.13,0.18,0.02,0.32])
parent2 = Individual([0.10,0.13,0.19,0.16,0.21,0.21])

NameError: ignored

In [0]:
def crossover(parent1, parent2 , list_t_max):
        num_of_gene = parent1.get_size()
        # t1 = sum(parent2.chromosome)
        # print(t1)
        # t2 = sum(parent1.chromosome)
        # print(t2)
        # cutting_point = random.randint(0,num_of_gene-1)
        chromosome1 = [None]*num_of_gene
        chromosome2 = [None]*num_of_gene
        cutting_point =3
        chromosome1[:cutting_point] = parent1.chromosome[:cutting_point]
        chromosome2[:cutting_point] = parent2.chromosome[:cutting_point]
        # beta = random.uniform(-0.5, 0.5)
        beta = 0.18/0.16 -1
        chromosome1[cutting_point] = (1-beta)*parent1.chromosome[cutting_point] + beta*parent2.chromosome[cutting_point]
        chromosome2[cutting_point] = (1-beta)*parent2.chromosome[cutting_point] + beta*parent1.chromosome[cutting_point]
        chromosome1[cutting_point+1:] = parent2.chromosome[cutting_point+1:]
        chromosome2[cutting_point+1:] = parent1.chromosome[cutting_point+1:]
        # for i in range(num_of_gene):
        #     # print(i)
        #     if parent1.chromosome[i] not in chromosome2:
        #         chromosome2.append(parent1.chromosome[i])
        #     if parent2.chromosome[i] not in chromosome1:
        #         chromosome1.append(parent2.chromosome[i])
        if sum(chromosome1) > 1:
            chromosome1 =[i/sum(chromosome1) for i in chromosome1]
            for i in range(len(chromosome2)-1):
                g = random.uniform(0,min(1-sum(chromosome2),list_t_max[i] -chromosome2[i]))
                chromosome2[i] += g
            chromosome2[-1] = 1 - sum(chromosome2[:-2])

        else:
            chromosome2 =[i/sum(chromosome2) for i in chromosome2]
            for i in range(len(chromosome1)-1):
                g = random.uniform(0,min(1-sum(chromosome1),list_t_max[i] - chromosome1[i]))
                chromosome1[i] += g
            chromosome1[-1] = 1 - sum(chromosome1[:-2])


        child1 = Individual(chromosome1)
        child2 = Individual(chromosome2)
        return child1, child2

In [0]:
# t1

In [87]:
a,b = crossover( parent1, parent2,[0.2,0.14,0.3,0.3,0.3,0.3])

asd


In [73]:
print(a.chromosome)
print(sum(a.chromosome))

[0.11136890951276103, 0.2134570765661253, 0.12064965197215778, 0.16473317865429235, 0.1948955916473318, 0.1948955916473318]
1.0


In [88]:
print(b.chromosome)
print(sum(b.chromosome))

[0.1420837210104195, 0.1397107637191077, 0.21113632944749428, 0.1629838665235641, 0.022352920989539487, 0.34408531929941444]
1.0223529209895394
